In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ns765\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],
    

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ','fick' ,' fux ', 'f\*\*',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck','fuk', 'wtf','fucck','f cking'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' asshole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'ass hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h','beetch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' transgender':
        [
            'transgender','trans gender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k','diick '
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bullshit ':
        [
            'bullsh\*t', 'bull\$hit','bs'
        ],

    ' homosexual':
        [
            'homo sexual','homosex'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots', 'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shithole ':
        [
            'shythole','shit hole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumbass':
        [
            'dumb ass', 'dubass'
        ],

    ' asshead':
        [
            'butthead', 'ass head'
        ],

    ' sex ':
        [
            's3x', 'sexuality',
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' motherfucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker', 'mother fucker'
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}

In [5]:
##Text Normalization

def clean_text(text,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):

  if is_lower:
    text=text.lower()
    
  if remove_patterns_text:
    for target, patterns in RE_PATTERNS.items():
      for pat in patterns:
        text=str(text).replace(pat, target)

  if remove_repeat_text:
    text = re.sub(r'(.)\1{2,}', r'\1', text) 

  text = str(text).replace("\n", " ")
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub('[0-9]',"",text)
  text = re.sub(" +", " ", text)
  text = re.sub("([^\x00-\x7F])+"," ",text)
  return text 

In [6]:
train['processed_comment_text'] = train['comment_text'].apply(lambda x:clean_text(x))

In [7]:
##Lemmatization

lemmatizer= WordNetLemmatizer()
def lemma(text, lemmatization=True):
  output=''
  if lemmatization:
    text=text.split(' ')
    for word in text:
      word1 = lemmatizer.lemmatize(word, pos = "n") #noun 
      word2 = lemmatizer.lemmatize(word1, pos = "v") #verb
      word3 = lemmatizer.lemmatize(word2, pos = "a") #adjective
      word4 = lemmatizer.lemmatize(word3, pos = "r") #adverb
      output=output + " " + word4
  else:
    output=text
  
  return str(output.strip())

In [8]:
train['processed_comment_text2'] = train['processed_comment_text'].apply(lambda x:lemma(x))

In [9]:
train_comments = train['processed_comment_text2'].fillna("_na_").values
print (train_comments[8])

sorry if the word nonsense wa offensive to you anyway i m not intend to write anything in the article wow they would jump on me for vandalism i m merely request that it be more encyclopedic so one can use it for school a a reference i have be to the selective breed page but it s almost a stub it point to animal breed which be a short messy article that give you no info there must be someone around with expertise in eugenics


In [10]:
train_y = train[ ['toxic' , 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate'] ].values
train_y[12]

array([1, 0, 0, 0, 0, 0], dtype=int64)

In [11]:
max_words = 30000
max_len = 100

In [12]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_comments)
train_comments_tokenized = tokenizer.texts_to_sequences(train_comments)
print (train_comments_tokenized[8])

[286, 23, 1, 164, 746, 25, 1332, 3, 8, 493, 5, 73, 14, 882, 3, 110, 238, 11, 1, 18, 1330, 52, 53, 1570, 15, 37, 12, 224, 5, 73, 983, 207, 9, 10, 2, 66, 1275, 39, 46, 35, 42, 10, 12, 402, 4, 4, 141, 5, 16, 2, 3, 1, 3781, 2337, 24, 27, 10, 17, 661, 4, 1229, 10, 127, 3, 1397, 2337, 59, 2, 4, 623, 5776, 18, 9, 123, 8, 48, 463, 41, 274, 2, 171, 334, 22, 2885, 11, 10508]


In [13]:
train_X = pad_sequences(train_comments_tokenized, maxlen=max_len, padding='post')
train_X[8]

array([  286,    23,     1,   164,   746,    25,  1332,     3,     8,
         493,     5,    73,    14,   882,     3,   110,   238,    11,
           1,    18,  1330,    52,    53,  1570,    15,    37,    12,
         224,     5,    73,   983,   207,     9,    10,     2,    66,
        1275,    39,    46,    35,    42,    10,    12,   402,     4,
           4,   141,     5,    16,     2,     3,     1,  3781,  2337,
          24,    27,    10,    17,   661,     4,  1229,    10,   127,
           3,  1397,  2337,    59,     2,     4,   623,  5776,    18,
           9,   123,     8,    48,   463,    41,   274,     2,   171,
         334,    22,  2885,    11, 10508,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [14]:
train_X.shape

(159571, 100)

In [15]:
word_index=tokenizer.word_index
print(len(word_index))
# unique
embedding_dim_fasttext = 300

149254


In [16]:
embeddings_index_fasttext = {}
f = open('E:\wiki-news-300d-1M.vec\wiki-news-300d-1M.vec', encoding='utf8')
for line in f:
    line.encode('utf-8').strip()
    values = line.split()
    word = values[0]
    embeddings_index_fasttext[word] = np.asarray(values[1:], dtype='float32')
f.close()
embedding_matrix_fasttext = np.random.random((len(word_index) + 1, embedding_dim_fasttext))
for word, i in word_index.items():
    embedding_vector = embeddings_index_fasttext.get(word)
    if embedding_vector is not None:
        embedding_matrix_fasttext[i] = embedding_vector

In [17]:
def BiLSTM():
    X_inp = Input( shape=(max_len,) )
    
    X = Embedding(max_words, 128)(X_inp)
    
    X = Bidirectional(LSTM(200, return_sequences=True))(X)
    
    X = Dropout(0.1)(X)
    
    X = LSTM(100, return_sequences=True)(X)
    
    X = GlobalAvgPool1D()(X)
    
    X = Dense(50, activation='relu')(X)
    
    X = Dropout(0.1)(X)
    
    X = Dense(6, activation='sigmoid')(X)
    
    model  = Model(inputs=X_inp, outputs=X)
    
    return model
    
    

In [18]:
toxicity_model1 = BiLSTM()

In [19]:

toxicity_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
toxicity_model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 128)          3840000   
                                                                 
 bidirectional (Bidirection  (None, 100, 400)          526400    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 400)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 100)          200400    
                                                                 
 global_average_pooling1d (  (None, 100)               0         
 GlobalAveragePooling1D)                                     

In [ ]:
batch_size = 64
epochs = 2
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')  
# for early stopping and preventing overfitting
toxicity_model1.fit(train_X,train_y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[es])
# data is divided into training and testing in 80:20 ratio

Epoch 1/2


 136/1995 [=>............................] - ETA: 23:14 - loss: 0.1744 - accuracy: 0.6247

In [ ]:
def predict(inp, model):
    inp_tokenized = tokenizer.texts_to_sequences([inp])
    final_inp = pad_sequences(inp_tokenized, maxlen=max_len, padding='post')
    print (model.predict(final_inp))
    print('  toxic    ' , 'severe_toxic ' , 'obscene   ' , 'threat   ' , '  insult' , '  identity_hate')
    

In [ ]:
predict("you are idiot", toxicity_model1)

In [ ]:
predict("hey you have done a great job ", toxicity_model1)

toxicity_model1.save_weights("toxicity_model2_weights.h5")

In [26]:
# import pickle

# with open('tokenizer.pk', 'wb') as tk:
#     pickle.dump(tokenizer, tk, protocol=pickle.HIGHEST_PROTOCOL)
